In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd "/content/drive/MyDrive/Spring_2023_Project/COMS_572_Source_Code/MNIST"

/content/drive/MyDrive/Spring_2023_Project/COMS_572_Source_Code/MNIST


In [3]:
## load package
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from keras.models import Sequential
from sklearn.model_selection import train_test_split

In [4]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [5]:
# load build-in dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

11490434/11490434 [==============================] - 0s 0us/step


In [6]:
## reshape and normalize data
x_train = x_train.reshape(60000, 28, 28, 1)
x_test = x_test.reshape(10000, 28, 28, 1)
y_train = y_train.reshape(60000,)
y_test = y_test.reshape(10000,)


In [7]:
y_train_one_hot = tf.keras.utils.to_categorical(y_train)
y_test_one_hot = tf.keras.utils.to_categorical(y_test)

In [8]:
train_arr_75 = [tf.image.grayscale_to_rgb(tf.image.resize(img, (75, 75))) for img in x_train]
train_arr_75 = np.array(train_arr_75)
train_arr_75.shape

(60000, 75, 75, 3)

In [9]:
test_arr_75 = [tf.image.grayscale_to_rgb(tf.image.resize(img, (75, 75))) for img in x_test]
test_arr_75 = np.array(test_arr_75)
test_arr_75.shape

(10000, 75, 75, 3)

In [10]:
model = tf.keras.models.load_model('epoch10_stage_3_MNIST_VGG_build_model.h5')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 2, 2, 512)         20024384  
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 256)               524544    
                                                                 
 batch_normalization (BatchN  (None, 256)              1024      
 ormalization)                                                   
                                                                 
 dense_1 (Dense)             (None, 256)               65792     
                                                                 
 batch_normalization_1 (Batc  (None, 256)              1024      
 hNormalization)                                        

In [11]:
model.layers[0].trainable=True
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 2, 2, 512)         20024384  
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 256)               524544    
                                                                 
 batch_normalization (BatchN  (None, 256)              1024      
 ormalization)                                                   
                                                                 
 dense_1 (Dense)             (None, 256)               65792     
                                                                 
 batch_normalization_1 (Batc  (None, 256)              1024      
 hNormalization)                                        

In [12]:
obtained_learning_rate = np.loadtxt('epoch10_stage_2_lr.txt')

In [13]:
def scheduler(epoch):
  return obtained_learning_rate[epoch]
callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [14]:
opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(
      loss=tf.keras.losses.CategoricalCrossentropy(),
      optimizer=opt,
      metrics=[tf.keras.metrics.CategoricalAccuracy()]
  )

In [ ]:
with tf.device('/device:GPU:0'):
  history2 = model.fit(train_arr_75, y_train_one_hot, validation_data=(test_arr_75, y_test_one_hot), epochs = 10, verbose=1, callbacks=[callback])

In [17]:
 # evaluate the model
 result = model.evaluate(test_arr_75, y_test_one_hot, verbose=1)

313/313 [==============================] - 8s 27ms/step - loss: 0.0239 - categorical_accuracy: 0.9922


In [18]:
result[1]*100

99.22000169754028

In [19]:
model.save('epoch10_10_final_MNIST_VGG_build_model.h5')